In [ ]:
# norfair dependencies
%cd /kaggle/input/norfair031py3/
!pip install commonmark-0.9.1-py2.py3-none-any.whl -f ./ --no-index
!pip install rich-9.13.0-py3-none-any.whl

!mkdir /kaggle/working/tmp
!cp -r /kaggle/input/norfair031py3/filterpy-1.4.5/filterpy-1.4.5/ /kaggle/working/tmp/
%cd /kaggle/working/tmp/filterpy-1.4.5/
!pip install .
!rm -rf /kaggle/working/tmp

# norfair
%cd /kaggle/input/norfair031py3/
!pip install norfair-0.3.1-py3-none-any.whl -f ./ --no-index
%cd ..

In [ ]:
kernel_mode = True
# kernel_mode = False
val_mode = False
# val_mode = True

save_video = False
# save_video = True
tracking = False
# tracking = True

import sys

if kernel_mode:
    sys.path.insert(0, '../input/weightedboxesfusion')
import warnings

warnings.filterwarnings("ignore")
import numpy as np
from tqdm.notebook import tqdm

tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import glob
import shutil

import torch
from PIL import Image
import ast
import gc

import subprocess

from ensemble_boxes import *

from norfair import Detection, Tracker

gc.enable()

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
if kernel_mode:
    lib_path = '/kaggle/input/yolov5-lib-ds'
else:
    lib_path = '/workspace/Github/yolov5'

# IMG_SIZE = 1280
IMG_SIZE = 2560
CONF = 0.4
# CONF = 0.15
IOU = 0.2
# CONF = 0.05
# CONF = 0.5
# IOU = 0.55
# IOU = 0.45
AUGMENT = True
TRACKING  = True

model_outputs = {
    "YOLOv5m6-f10-v8-2560-rand-crop-fold0": {
        "config_name":
        "yolov5m",
        "model_name":
        "yolov5m6",
        "model_file":
        "../input/yolov5m6-f10-v8-2560-rand-crop-fold0/best.pt"
        if kernel_mode else
        "/workspace/Github/yolov5/starfish/YOLOv5m6-f10-v8-2560-rand-crop-fold0/weights/best.pt",
        "CONF":
        CONF,
        "IOU":
        IOU,
        "IMG_SIZE":
        IMG_SIZE,
        "weight":
        1.0,
    },
    "YOLOv5m6-f10-v8-2560-rand-crop-fold1": {
        "config_name":
        "yolov5m",
        "model_name":
        "yolov5m6",
        "model_file":
        "../input/yolov5m6-f10-v8-2560-rand-crop-fold1/best.pt"
        if kernel_mode else
        "/workspace/Github/yolov5/starfish/YOLOv5m6-f10-v8-2560-rand-crop-fold1/weights/best.pt",
        "CONF":
        CONF,
        "IOU":
        IOU,
        "IMG_SIZE":
        IMG_SIZE,
        "weight":
        1.0,
    },
    "YOLOv5m6-f10-v8-2560-rand-crop-fold2": {
        "config_name":
        "yolov5m",
        "model_name":
        "yolov5m6",
        "model_file":
        "../input/yolov5m6-f10-v8-2560-rand-crop-fold2/best.pt"
        if kernel_mode else
        "/workspace/Github/yolov5/starfish/YOLOv5m6-f10-v8-2560-rand-crop-fold2/weights/best.pt",
        "CONF":
        CONF,
        "IOU":
        IOU,
        "IMG_SIZE":
        IMG_SIZE,
        "weight":
        1.0,
    },
    "YOLOv5m6-f10-v8-2560-rand-crop-fold3": {
        "config_name":
        "yolov5m",
        "model_name":
        "yolov5m6",
        "model_file":
        "../input/yolov5m6-f10-v8-2560-rand-crop-fold3/best.pt"
        if kernel_mode else
        "/workspace/Github/yolov5/starfish/YOLOv5m6-f10-v8-2560-rand-crop-fold3/weights/best.pt",
        "CONF":
        CONF,
        "IOU":
        IOU,
        "IMG_SIZE":
        IMG_SIZE,
        "weight":
        1.0,
    },
    "YOLOv5m6-f10-v8-2560-rand-crop-fold4": {
        "config_name":
        "yolov5m",
        "model_name":
        "yolov5m6",
        "model_file":
        "../input/yolov5m6-f10-v8-2560-rand-crop-fold4/best.pt"
        if kernel_mode else
        "/workspace/Github/yolov5/starfish/YOLOv5m6-f10-v8-2560-rand-crop-fold4/weights/best.pt",
        "CONF":
        CONF,
        "IOU":
        IOU,
        "IMG_SIZE":
        IMG_SIZE,
        "weight":
        1.0,
    },
    "YOLOv5m6-f10-v8-2560-rand-crop-fold5": {
        "config_name":
        "yolov5m",
        "model_name":
        "yolov5m6",
        "model_file":
        "../input/yolov5m6-f10-v8-2560-rand-crop-fold5/best.pt"
        if kernel_mode else
        "/workspace/Github/yolov5/starfish/YOLOv5m6-f10-v8-2560-rand-crop-fold5/weights/best.pt",
        "CONF":
        CONF,
        "IOU":
        IOU,
        "IMG_SIZE":
        IMG_SIZE,
        "weight":
        1.0,
    },
    "YOLOv5m6-f10-v8-2560-rand-crop-fold6": {
        "config_name":
        "yolov5m",
        "model_name":
        "yolov5m6",
        "model_file":
        "../input/yolov5m6-f10-v8-2560-rand-crop-fold6/best.pt"
        if kernel_mode else
        "/workspace/Github/yolov5/starfish/YOLOv5m6-f10-v8-2560-rand-crop-fold6/weights/best.pt",
        "CONF":
        CONF,
        "IOU":
        IOU,
        "IMG_SIZE":
        IMG_SIZE,
        "weight":
        1.0,
    },
    "YOLOv5m6-f10-v8-2560-rand-crop-fold7": {
        "config_name":
        "yolov5m",
        "model_name":
        "yolov5m6",
        "model_file":
        "../input/yolov5m6-f10-v8-2560-rand-crop-fold7/best.pt"
        if kernel_mode else
        "/workspace/Github/yolov5/starfish/YOLOv5m6-f10-v8-2560-rand-crop-fold7/weights/best.pt",
        "CONF":
        CONF,
        "IOU":
        IOU,
        "IMG_SIZE":
        IMG_SIZE,
        "weight":
        1.0,
    },
    "YOLOv5m6-f10-v8-2560-rand-crop-fold8": {
        "config_name":
        "yolov5m",
        "model_name":
        "yolov5m6",
        "model_file":
        "../input/yolov5m6-f10-v8-2560-rand-crop-fold8/best.pt"
        if kernel_mode else
        "/workspace/Github/yolov5/starfish/YOLOv5m6-f10-v8-2560-rand-crop-fold8/weights/best.pt",
        "CONF":
        CONF,
        "IOU":
        IOU,
        "IMG_SIZE":
        IMG_SIZE,
        "weight":
        1.0,
    },
    "YOLOv5m6-f10-v8-2560-rand-crop-fold9": {
        "config_name":
        "yolov5m",
        "model_name":
        "yolov5m6",
        "model_file":
        "../input/yolov5m6-f10-v8-2560-rand-crop-fold9/best.pt"
        if kernel_mode else
        "/workspace/Github/yolov5/starfish/YOLOv5m6-f10-v8-2560-rand-crop-fold9/weights/best.pt",
        "CONF":
        CONF,
        "IOU":
        IOU,
        "IMG_SIZE":
        IMG_SIZE,
        "weight":
        1.0,
    },
}

# colors = [(0, 0, 130), (0, 130, 0), (255, 0, 0), (255, 0, 255), (0, 0, 255)]
np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255)) \
          for idx in range(len(model_outputs))]

iou_thr = 0.55
# skip_box_thr = 0.05
# skip_box_thr = 0.01
# skip_box_thr = 0.0005
skip_box_thr = 0.0001
sigma = 0.01
weights = np.ones(len(model_outputs))

# Utility Functions

In [ ]:
def voc2yolo(bboxes, image_height=720, image_width=1280):
    """
    voc  => [x1, y1, x2, y1]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    bboxes = bboxes.copy().astype(
        float)  # otherwise all value will be 0 as voc_pascal dtype is np.int
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]] / image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]] / image_height
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    bboxes[..., 0] = bboxes[..., 0] + w / 2
    bboxes[..., 1] = bboxes[..., 1] + h / 2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    return bboxes


def yolo2voc(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """
    bboxes = bboxes.copy().astype(
        float)  # otherwise all value will be 0 as voc_pascal dtype is np.int
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]] * image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]] * image_height
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]] / 2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    return bboxes


def coco2yolo(bboxes, image_height=720, image_width=1280):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    bboxes = bboxes.copy().astype(
        float)  # otherwise all value will be 0 as voc_pascal dtype is np.int
    # normolizinig
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]] / image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]] / image_height
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]] / 2
    return bboxes


def yolo2coco(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    """
    bboxes = bboxes.copy().astype(
        float)  # otherwise all value will be 0 as voc_pascal dtype is np.int
    # denormalizing
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]] * image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]] * image_height
    # converstion (xmid, ymid) => (xmin, ymin)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]] / 2
    return bboxes


def voc2coco(bboxes, image_height=720, image_width=1280):
    bboxes = voc2yolo(bboxes, image_height, image_width)
    bboxes = yolo2coco(bboxes, image_height, image_width)
    return bboxes


def load_image(image_path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)


def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(
        0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img,
                    label, (c1[0], c1[1] - 2),
                    0,
                    tl / 3, [225, 255, 255],
                    thickness=tf,
                    lineType=cv2.LINE_AA)


def draw_bboxes(img,
                bboxes,
                classes,
                class_ids,
                colors=None,
                show_classes=None,
                bbox_format='yolo',
                class_name=False,
                line_thickness=2):

    image = img.copy()
    show_classes = classes if show_classes is None else show_classes
    colors = (0, 255, 0) if colors is None else colors

    if bbox_format == 'yolo':

        for idx in range(len(bboxes)):

            bbox = bboxes[idx]
            cls = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:

                x1 = round(float(bbox[0]) * image.shape[1])
                y1 = round(float(bbox[1]) * image.shape[0])
                w = round(float(bbox[2]) * image.shape[1] / 2)  #w/2
                h = round(float(bbox[3]) * image.shape[0] / 2)

                voc_bbox = (x1 - w, y1 - h, x1 + w, y1 + h)
                plot_one_box(voc_bbox,
                             image,
                             color=color,
                             label=cls if class_name else str(get_label(cls)),
                             line_thickness=line_thickness)

    elif bbox_format == 'coco':

        for idx in range(len(bboxes)):

            bbox = bboxes[idx]
            cls = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                w = int(round(bbox[2]))
                h = int(round(bbox[3]))

                voc_bbox = (x1, y1, x1 + w, y1 + h)
                plot_one_box(voc_bbox,
                             image,
                             color=color,
                             label=cls if class_name else str(cls_id),
                             line_thickness=line_thickness)

    elif bbox_format == 'voc_pascal':

        for idx in range(len(bboxes)):

            bbox = bboxes[idx]
            cls = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                x2 = int(round(bbox[2]))
                y2 = int(round(bbox[3]))
                voc_bbox = (x1, y1, x2, y2)
                plot_one_box(voc_bbox,
                             image,
                             color=color,
                             label=cls if class_name else str(cls_id),
                             line_thickness=line_thickness)
    else:
        raise ValueError('wrong bbox format')

    return image


def show_img(img, bboxes, bbox_format='yolo', bbox_colors=None):
    names = ['starfish'] * len(bboxes)
    labels = [0] * len(bboxes)
    img = draw_bboxes(img=img,
                      bboxes=bboxes,
                      classes=names,
                      class_ids=labels,
                      class_name=True,
                      colors=colors if bbox_colors is None else bbox_colors,
                      bbox_format=bbox_format,
                      line_thickness=2)
    return Image.fromarray(img).resize((800, 400))


def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes


def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row


# Helper to convert bbox in format [x_min, y_min, x_max, y_max, score] to norfair.Detection class
def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(
            Detection(points=np.array([xc, yc]),
                      scores=np.array([score]),
                      data=np.array([w, h, frame_id])))

    return result


# Euclidean distance function to match detections on this frame with tracked_objects from previous frames
def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)


np.random.seed(32)
# colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
#           for idx in range(1)]

In [ ]:
def load_model(ckpt_path,
               lib_path='/kaggle/input/yolov5-lib-ds',
               conf=0.25,
               iou=0.50):
    model = torch.hub.load(lib_path,
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou = iou  # NMS IoU threshold
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 30  # maximum number of detections per image
    return model


def predict(model, img, size=768, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds = results.pandas().xyxy[0]
    bboxes = preds[['xmin', 'ymin', 'xmax', 'ymax']].values
    if len(bboxes):
        bboxes = voc2coco(bboxes, height, width).astype(int)
        confs = preds.confidence.values
        classes = preds.name.values
        return bboxes, confs, classes
    else:
        return [], [], []


def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes) > 0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot += ' '
        annot = annot.strip(' ')
    return annot

In [ ]:
def run_wbf(bboxes,
            confs,
            classs,
            image_size,
            iou_thr=0.50,
            skip_box_thr=0.0001,
            weights=None):
    
    boxes = [bbox / (image_size - 1) for bbox in bboxes]
    scores = [conf for conf in confs]
    labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = weighted_boxes_fusion(boxes,
                                                  scores,
                                                  labels,
                                                  weights=weights,
                                                  iou_thr=iou_thr,
                                                  skip_box_thr=skip_box_thr)
    boxes = boxes * (image_size - 1)
    return boxes, scores, labels

In [ ]:
def tracking_function(tracker, frame_id, bboxes, scores):

    detects = []
    scores = []
    predictions = []

    if len(scores) > 0:
        for i in range(len(bboxes)):
            box = bboxes[i]
            score = scores[i]
            x_min = int(box[0])
            y_min = int(box[1])
            bbox_width = int(box[2])
            bbox_height = int(box[3])
            detects.append(
                [x_min, y_min, x_min + bbox_width, y_min + bbox_height, score])
            predictions.append('{:.2f} {} {} {} {}'.format(
                score, x_min, y_min, bbox_width, bbox_height))
            scores.append(score)

            # print(predictions[:-1])

    # Update tracks using detects from current frame
    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id))
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data

        if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
            continue

        # Add objects that have no detections on current frame to predictions
        xc, yc = tobj.estimate[0]
        x_min, y_min = int(round(xc - bbox_width / 2)), int(
            round(yc - bbox_height / 2))
        score = tobj.last_detection.scores[0]

        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min,
                                                       bbox_width,
                                                       bbox_height))
        scores.append(score)

    return predictions, scores

# Ensemble Validation Test

In [ ]:
if val_mode:
    if kernel_mode:
        TEST_IMAGE_PATH = "/kaggle/input/tensorflow-great-barrier-reef/train_images/video_2/5747.jpg"
        # TEST_IMAGE_PATH = "/kaggle/input/tensorflow-great-barrier-reef/train_images/video_1/5659.jpg"
    else:
        TEST_IMAGE_PATH = "/workspace/Kaggle/Starfish/train_images/video_2/5747.jpg"
        # TEST_IMAGE_PATH = "/workspace/Kaggle/Starfish/train_images/video_1/5659.jpg"

    img = cv2.imread(TEST_IMAGE_PATH)

    out_image = None

    boxes_list = []
    scores_list = []
    labels_list = []
    weight_list = []
    for i, (model_name, metadata) in enumerate(model_outputs.items()):
        print(model_name)
        ckpt_path = metadata["model_file"]
        weight = metadata["weight"]

        model = load_model(ckpt_path, lib_path=lib_path, conf=0.4, iou=0.2)

        bboxes, scores, bbclasses = predict(model,
                                            img,
                                            size=2560,
                                            augment=AUGMENT)

        if out_image is not None:
            out_image = draw_bboxes(img=out_image,
                                    bboxes=bboxes,
                                    classes=[f"model{i+1}"] * len(bboxes),
                                    class_ids=[0] * len(bboxes),
                                    class_name=True,
                                    colors=colors[i],
                                    bbox_format='coco',
                                    line_thickness=2)
        else:
            out_image = draw_bboxes(img=img,
                                    bboxes=bboxes,
                                    classes=[f"model{i+1}"] * len(bboxes),
                                    class_ids=[0] * len(bboxes),
                                    class_name=True,
                                    colors=colors[i],
                                    bbox_format='coco',
                                    line_thickness=2)

        if len(bboxes) > 0:
            # Convert (h, w) to (x2, y2)
            bboxes[:, 2] = bboxes[:, 2] + bboxes[:, 0]
            bboxes[:, 3] = bboxes[:, 3] + bboxes[:, 1]

            boxes_list.append(bboxes)
            scores_list.append(scores)
            labels_list.append(bbclasses)
            weight_list.append(weight)

        del model
        torch.cuda.empty_cache()
        gc.collect()

    out_image = cv2.cvtColor(out_image, cv2.COLOR_BGR2RGB)
    display(Image.fromarray(out_image))

In [ ]:
if val_mode:
    ## WBF ##
    boxes, scores, labels = run_wbf(boxes_list,
                                    scores_list,
                                    labels_list,
                                    IMG_SIZE,
                                    iou_thr,
                                    skip_box_thr,
                                    weights=weight_list)
    # Convert (x2, y2) back to (h, w)
    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
    out_image_esemble = draw_bboxes(img=img,
                                    bboxes=boxes,
                                    classes=["WBF"] * len(boxes),
                                    class_ids=[0] * len(boxes),
                                    class_name=True,
                                    colors=(50, 50, 50),
                                    bbox_format='coco',
                                    line_thickness=2)
    out_image_esemble = cv2.cvtColor(out_image_esemble, cv2.COLOR_BGR2RGB)
    print(
        "------------------------------  WBF   ---------------------------------------"
    )
    display(Image.fromarray(out_image_esemble))

# Ensemble Video

In [ ]:
if save_video:
    TRAIN_PATH = '/workspace/Kaggle/Starfish'
    df = pd.read_csv(f"{TRAIN_PATH}/train.csv")

    def get_path(row):
        row['image_path'] = f'{TRAIN_PATH}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
        return row

    df['annotations'] = df['annotations'].apply(eval)
    df['n_annotations'] = df['annotations'].str.len()
    df['has_annotations'] = df['annotations'].str.len() > 0
    df['has_2_or_more_annotations'] = df['annotations'].str.len() >= 2
    df['doesnt_have_annotations'] = df['annotations'].str.len() == 0
    df['start_cut_here'] = df['has_annotations'] & df[
        'doesnt_have_annotations'].shift(
            1) & df['doesnt_have_annotations'].shift(2)
    df['end_cut_here'] = df['doesnt_have_annotations'] & df[
        'has_annotations'].shift(1) & df['has_annotations'].shift(2)
    df['sequence_change'] = df['sequence'] != df['sequence'].shift(1)
    df['last_row'] = df.index == len(df) - 1
    df['cut_here'] = df['start_cut_here'] | df['end_cut_here'] | df[
        'sequence_change'] | df['last_row']
    start_idx = 0
    for subsequence_id, end_idx in enumerate(df[df['cut_here']].index):
        df.loc[start_idx:end_idx, 'subsequence_id'] = subsequence_id
        start_idx = end_idx
    df['subsequence_id'] = df['subsequence_id'].astype(int)
    drop_cols = [
        'start_cut_here', 'end_cut_here', 'sequence_change', 'last_row',
        'cut_here', 'has_2_or_more_annotations', 'doesnt_have_annotations'
    ]
    df = df.drop(drop_cols, axis=1)

    # Path of images
    df = df.progress_apply(get_path, axis=1)

    df_manybbox = df[df["n_annotations"] > 3]

In [ ]:
# if tracking:
#     tracker = Tracker(
#         distance_function=euclidean_distance,
#         distance_threshold=30,
#         hit_inertia_min=3,
#         hit_inertia_max=6,
#         initialization_delay=1,
#     )

# model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
# image_paths = df[df.num_bbox > 1].sample(100).image_path.tolist()
# frame_id = 0
# for idx, path in enumerate(image_paths):
#     img = cv2.imread(path)[..., ::-1]
#     if FDA_aug:
#         img = FDA_trans(image=img)['image']
#     bboxes, confis = predict(model, img, size=IMG_SIZE, augment=AUGMENT)
#     predict_box = tracking_function(tracker, frame_id, bboxes, confis)

#     if len(predict_box) > 0:
#         box = [list(map(int, box.split(' ')[1:])) for box in predict_box]
#     else:
#         box = []
#     display(show_img(img, box, bbox_format='coco'))
#     if idx > 5:
#         break
        
#     frame_id += 1

In [ ]:
if save_video:
    # slice_limit = 50
    slice_limit = 300
    loaded_models = {}

    selected_slices = {
        0: 5250,
        1: 4500,
        2: 5650,
        # 2: 5741,
        # 2: 4750,
    }
    selected_fps = {
        0: 10,
        1: 5,
        2: 10,
    }
    for video_id in [2]:
        #     for video_id in df["video_id"].unique():
        # partly borrowed from https://github.com/RobMulla/helmet-assignment/blob/main/helmet_assignment/video.py
        #         fps = 30
        #         width = 1280
        #         height = 720

        if tracking:
            save_path = f'video_{video_id}_track.mp4'
            tmp_path = f'tmp_video_{video_id}_track.avi'
        else:
            save_path = f'video_{video_id}.mp4'
            tmp_path = f'tmp_video_{video_id}.avi'
            #         output_video = cv2.VideoWriter(tmp_path,
            #                                        cv2.VideoWriter_fourcc(*"MP4V"), fps,
            #                                        (width, height))

        video_size = (1280, 720)

        output_video = cv2.VideoWriter(tmp_path,
                                       cv2.VideoWriter_fourcc(*'DIVX'),
                                       selected_fps[video_id], video_size)

        video_df = df[df["video_id"] == video_id]

        # slice_start = np.random.randint(df_manybbox.shape[0] - slice_limit)
        # slice_start = np.random.randint(df_manybbox.shape[0] - slice_limit)
        # slice_start = np.random.choice(df_manybbox.index, size=1)[0]
        # slice_start = 5464
        slice_start = selected_slices[video_id]
        print(f"Slice start: {slice_start}")

        slice_df = video_df[video_df["video_frame"] >= slice_start].head(
            slice_limit)
        # slice_df = video_df.iloc[slice_start:slice_start + slice_limit, :]

        frame_id = 0
        tracker = Tracker(
            distance_function=euclidean_distance,
            distance_threshold=30,
            hit_inertia_min=3,
            hit_inertia_max=6,
            initialization_delay=1,
        )

        for index, row in tqdm(slice_df.iterrows(), total=slice_df.shape[0]):

            img_path = row["image_path"]

            img = cv2.imread(img_path)

            colors = [(0, 0, 130), (0, 130, 0), (255, 0, 0), (255, 0, 255),
                      (0, 0, 255)]

            boxes_list = []
            scores_list = []
            labels_list = []
            weight_list = []
            for i, (model_name, metadata) in enumerate(model_outputs.items()):
                # print(model_name)
                ckpt_path = metadata["model_file"]

                conf = metadata["CONF"]
                iou = metadata["IOU"]
                img_size = metadata["IMG_SIZE"]

                weight = metadata["weight"]

                if model_name not in loaded_models:
                    model = load_model(ckpt_path,
                                       lib_path=lib_path,
                                       conf=conf,
                                       iou=iou)
                    loaded_models[model_name] = model
                else:
                    model = loaded_models[model_name]

                bboxes, scores, bbclasses = predict(model,
                                                    img,
                                                    size=img_size,
                                                    augment=AUGMENT)

                if tracking:
                    predict_boxes, predict_scores = tracking_function(
                        tracker, frame_id, bboxes, scores)

                    if len(predict_boxes) > 0:
                        bboxes = [
                            list(map(int,
                                     box.split(' ')[1:]))
                            for box in predict_boxes
                        ]
                        bboxes = np.array(bboxes)
                        scores = np.array(predict_scores)

                if len(bboxes) > 0:
                    # Convert (h, w) to (x2, y2)
                    bboxes[:, 2] = bboxes[:, 2] + bboxes[:, 0]
                    bboxes[:, 3] = bboxes[:, 3] + bboxes[:, 1]

                    boxes_list.append(bboxes)
                    scores_list.append(scores)
                    labels_list.append(bbclasses)
                    weight_list.append(weight)

            ## WBF ##
            if len(boxes_list) > 0:
                ensembled_boxes, ensembled_scores, ensembled_labels = run_wbf(
                    boxes_list,
                    scores_list,
                    labels_list,
                    IMG_SIZE,
                    iou_thr,
                    skip_box_thr,
                    weights=weight_list)
                # Convert (x2, y2) back to (h, w)
                ensembled_boxes[:,
                                2] = ensembled_boxes[:, 2] - ensembled_boxes[:,
                                                                             0]
                ensembled_boxes[:,
                                3] = ensembled_boxes[:, 3] - ensembled_boxes[:,
                                                                             1]
                out_image_esemble = draw_bboxes(
                    img=img,
                    bboxes=ensembled_boxes,
                    classes=[
                        f"Starfish ({ensembled_scores[k]:.2f})"
                        for k in range(len(ensembled_boxes))
                    ],
                    class_ids=[0] * len(ensembled_boxes),
                    class_name=True,
                    colors=colors[1],
                    bbox_format='coco',
                    line_thickness=2)
            else:
                out_image_esemble = img

            # display(Image.fromarray(out_image_esemble))
            output_video.write(out_image_esemble)

            frame_id += 1

        output_video.release()

        # Not all browsers support the codec, we will re-load the file at tmp_output_path
        # and convert to a codec that is more broadly readable using ffmpeg
        if os.path.exists(save_path):
            os.remove(save_path)

        AVI2MP4 = "-ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4"

        # command = f"ffmpeg -i {tmp_path} {AVI2MP4} {save_path}"
        command = f"ffmpeg -i {tmp_path} -vf scale=1280:720 -preset slow {save_path}"
        subprocess.call(command, shell=True)
        #         subprocess.run([
        #             "ffmpeg", "-i", tmp_path, "-crf", "18", "-preset", "veryfast",
        #             "-vcodec", "libx264", save_path
        #         ])
        os.remove(tmp_path)

        break

In [ ]:
# Slice start: 5464

In [ ]:
from IPython.display import HTML
from base64 import b64encode


def play(filename):
    html = ''
    video = open(filename, 'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=800 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src
    return HTML(html)


# play('video_0.mp4')
# play('video_1.mp4')
# play('video_2.mp4')
# play('video_2_track.mp4')

# Ensemble Test Set


In [ ]:
%cd /kaggle/working/

In [ ]:
if kernel_mode:
    import greatbarrierreef
    env = greatbarrierreef.make_env()  # initialize the environment
    iter_test = env.iter_test()

    loaded_models = {}

    frame_id = 0
    tracker = Tracker(
        distance_function=euclidean_distance,
        distance_threshold=30,
        hit_inertia_min=3,
        hit_inertia_max=6,
        initialization_delay=1,
    )
    for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
        boxes_list = []
        scores_list = []
        labels_list = []
        weight_list = []
        for i, (model_name, metadata) in enumerate(model_outputs.items()):
            print(model_name)
            ckpt_path = metadata["model_file"]

            conf = metadata["CONF"]
            iou = metadata["IOU"]
            img_size = metadata["IMG_SIZE"]

            weight = metadata["weight"]

            if model_name not in loaded_models:
                model = load_model(ckpt_path,
                                   lib_path=lib_path,
                                   conf=conf,
                                   iou=iou)
                loaded_models[model_name] = model
            else:
                model = loaded_models[model_name]

            bboxes, scores, bbclasses = predict(model,
                                                img,
                                                size=img_size,
                                                augment=AUGMENT)

            try:
                if tracking:
                    predict_boxes, predict_scores = tracking_function(
                        tracker, frame_id, bboxes, scores)

                    if len(predict_boxes) > 0:
                        bboxes = [
                            list(map(int,
                                     box.split(' ')[1:]))
                            for box in predict_boxes
                        ]
                        bboxes = np.array(bboxes)
                        scores = np.array(predict_scores)
                
                if len(bboxes) > 0:
                    # Convert (h, w) to (x2, y2)
                    bboxes[:, 2] = bboxes[:, 2] + bboxes[:, 0]
                    bboxes[:, 3] = bboxes[:, 3] + bboxes[:, 1]

                    boxes_list.append(bboxes)
                    scores_list.append(scores)
                    labels_list.append(bbclasses)
                    weight_list.append(weight)

                if idx != 0 and idx % 1000 == 0:
                    # del model
                    # del loaded_models[model_name]
                    torch.cuda.empty_cache()
                    gc.collect()
            except:
                pass

        if len(boxes_list) > 0:
            try:
                ensembled_boxes, ensembled_scores, ensembled_labels = run_wbf(
                    boxes_list,
                    scores_list,
                    labels_list,
                    IMG_SIZE,
                    iou_thr,
                    skip_box_thr,
                    weights=weight_list)

                # Convert (x2, y2) back to (h, w)
                ensembled_boxes[:, 2] = ensembled_boxes[:, 2] - ensembled_boxes[:, 0]
                ensembled_boxes[:, 3] = ensembled_boxes[:, 3] - ensembled_boxes[:, 1]

                annotations = format_prediction(ensembled_boxes,
                                                ensembled_scores)
                pred_df['annotations'] = annotations
                env.predict(pred_df)
                
                if idx<3:
                    display(show_img(img, ensembled_boxes, bbox_format='coco'))
            except:
                pred_df['annotations'] = ''
                env.predict(pred_df)
        else:
            ensembled_boxes = []
            ensembled_scores = None
            pred_df['annotations'] = ''
            env.predict(pred_df)
            
        frame_id += 1

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()